# Automating Files Manipulation and Email Sending With Python

* The present project has the goal of facilitating manipulation of large groups of files.

* We are going to simulate a situation where we work for a company that manages different shopping malls spread through Brazil's territoty. In each state the company operates (Sao Paulo, Amazonas, Minas Gerais, Goias and Rio de Janeiro), there is an employee who is going to be responsible for analyzing the reports we are about to send.

* Thus, our task is to guarantee each employee gets the reports from the shopping malls that are present in his/her region (eg: the person responsible for analyzing the shopping malls from Sao Paulo must receive only the files that are related to Sao Paulo shopping malls).

In [2]:
# Apagar depois:
# Bibliotecas do projeto
from pathlib import Path
import shutil
import os
import datetime
from zipfile import ZipFile
import re

In [1]:
# Let's get started with our challenge
# Firstly, we ought to create a new directory made specially for this occasion
from pathlib import Path
import shutil
desafio = Path.cwd() / 'Desafio'
desafio.mkdir()

In [4]:
# Now, let's move this file into the new directory
file =  Path.cwd() / 'desafio_email_1.ipynb'
shutil.move(str(file), desafio)

'/Users/felipeveiga/Documents/Jupyter USP/Python Lira/pathlib e shutil/Desafio/desafio_email_1.ipynb'

In [58]:
# The shopping mall's files have been sent to my email address, so I'll have to move them into my current directory
from imap_tools import MailBox, AND
import os
from zipfile import ZipFile

# To do this I will have to log in my email address
username = os.environ.get('EMAIL_USER')
password = os.environ.get('MOT_USER')

# Let's log in
gmail = MailBox('imap.gmail.com').login(username, password)

# I will face the following issue: Iterating over my whole mail box would take a lot of time
# So, aspiring efficiency, I will have to create an algorithm that filters our search
# The first parameter I could give to the code is the date the message we are looking for was sent
# This will be sufficient to shrink my search range
import datetime
hoje = datetime.datetime.now().date()
emails_procurados = gmail.fetch(AND(date = hoje))

# With our narrowed search scope, let's look for the email containing what we seek
for email in emails_procurados:
    # The email I am looking for must have an attachment with a .zip extension
    # This zip file contains the shopping malls reports we are going to work on
    if len(email.attachments) != 0:
        for att in email.attachments:
            # This is a guarantee the file we are downloading has the zip extension
            if '.zip' in att.filename:
                print(att.filename)
                # Having found the right file, let's download it!
                with open('Shopping_Malls.zip', 'wb') as z:
                    byte = att.payload
                    z.write(byte)
     

Desafio_email1.zip


In [11]:
# Deletando os arquivos csv
# APAGAR DEPOIS!

current = Path.cwd()
for file in current.iterdir():
    if file.suffix == '.csv':
        os.remove(file)

In [43]:
# Now that we have just downloaded the zip file, we must extract its content.
with ZipFile('Shopping_Malls.zip', 'r') as zip:
    zip.extractall()
    
# Now that we got all the files we need, let's check the employees who need to receive them
from pandas import read_excel
employees = read_excel('Relação_Funcionarios.xlsx', header=1, usecols=[1,2,3])
employees

,Employee,Email,State
0,André Teixeira,teixeira.andre2110@gmail.com,AM
1,Pedro Alves,alves.pedro2110@gmail.com,GO
2,Paula Gonçalves,goncavespaula2110@yahoo.com,MG
3,Antônio Nunez,nunez.antonio2110@yahoo.com,RJ
4,João Traoré,traore.joao2110@yahoo.com,SP


In [44]:
# Let's agroup all the employee's states in a list
states = [state for state in employees['State']]
states

['AM', 'GO', 'MG', 'RJ', 'SP']

In [35]:
# With the .csv files in hand, we ought to agroup the files according to its respective state
# To make the sending easier  we are going to create a zip file containing, all the csv files of a certain state
current_directory = Path.cwd()
os.chdir(current_directory)

# In order to do this, we are going to iterate over the 'states' list we've just created

for state in states:
    # Creating a zip file for every state in the list
    with ZipFile(f'{state}.zip', 'w') as zip_file:
        # Analyzing each shopping mall csv file in our directory and checking its respective state
        # Obs: The state of the shopping mall is registered right in the file name
        for file in os.listdir(current_directory):
            # Checking if the file has a csv extension
            if '.csv' in file:
                # Checking if its state corresponds to the state in effect in the loop
                if state in file:
                    # If the two conditions are satisfied, Python will register the file in the zip file
                    zip_file.write(file)

# Now that the files are agrouped in their respective zip file, we don't need them anymore in our directory
for file in current_directory.iterdir():
    if file.suffix == '.csv':
        os.remove(file)

In [47]:
# Sending the files
# Now that we have agrouped the files in a zip according to their shoppping mall state, let's send them
                                                                            # to ther respective employee!


# We are going to cluster the informations of each row of the 'employees' DataFrame into a tuple
informations = zip(employees['Employee'], employees['Email'], employees['State'])

# It is time to send the files
import smtplib 
from mimetypes import guess_type
from email.message import EmailMessage
from getpass import getpass

current_directory = Path.cwd()
my_email = os.environ.get('EMAIL_USER')
password = getpass('Insert your password')

for information in informations:
    msg = EmailMessage()
    msg['From'] = my_email
    msg['To'] = information[1]  # information[1] is equivalent to the employee's email in our clustering
    msg['Subject'] = 'Shopping Malls reports'
    msg.set_content(f''' Dear {information[0]}
Here are the reports of the shopping malls from your state ({information[2]}).
Have a nice day!
Felipe Veiga''')
    for file in current_directory.iterdir():
        if information[2] in file.name:
            with open(file, 'rb') as f:
                file_content = f.read()
                maintype, subtype = guess_type(file)[0].split('/')
                msg.add_attachment(file_content, maintype=maintype, subtype=subtype, filename=file.name)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(my_email, password)
        smtp.send_message(msg)
            

Insert your password ········


In [31]:
print('''Dear Felipe
Here are the reports of the shopping malls in your state ''')

Dear Felipe
Here are the reports of the shopping malls in your state 
